In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/Emotion Recogition/code/python_files

/content/drive/My Drive/Emotion Recogition/code/python_files


In [3]:
import numpy as np
import os
import sys
import pandas as pd

import wave

import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten,Dropout,GlobalAveragePooling2D,MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from scipy import signal
import matplotlib.pyplot as plot
from helper import *
from sklearn.preprocessing import LabelEncoder
from keras import backend as K


Using TensorFlow backend.


In [0]:

code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/../data/sessions/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000

In [0]:
import pickle
with open(code_path + '/../data/'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [0]:
X_train = []
X_test = []
Y_train = []
Y_test = []
fs = 16e3
counter = 0
counter1 = 0
for ses_mod in data2:
    if 'impro' in ses_mod['id'] and ses_mod['emotion'] in emotions_used:
        f, t, Sxx = signal.spectrogram(ses_mod['signal'], fs, nperseg=400,noverlap=200)
        Sxx = pad_sequence_into_array(Sxx,maxlen=300,value=0)
        
        if ses_mod['id'][:5]=="Ses05":
            counter+=1
            X_test.append(Sxx)
            Y_test.append(ses_mod['id'][-4])
        else:
            counter1+=1
            X_train.append(Sxx)
            Y_train.append(ses_mod['id'][-4])
        
print(counter)
print(counter1)

X_test = np.array(X_test)
X_train = np.array(X_train)
print(X_train.shape)
print(X_test.shape)

625
2034
(2034, 201, 300)
(625, 201, 300)


In [0]:
X_train = X_train.reshape(-1,201,300,1)
X_test = X_test.reshape(-1,201,300,1)

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train=le.fit_transform(Y_train)
y_test=le.transform(Y_test)

In [0]:
def make_model():
    in_layer = Input(shape=(201, 300,1))
    x = Conv2D(32,(5,5), activation = 'elu')(in_layer)  
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(128, (5,5))(x)                           
    x = GlobalAveragePooling2D()(x)                     
    x = Dense(64,activation='elu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation = "sigmoid")(x) 
    model = Model(inputs = in_layer, outputs=output_layer)
    return model

In [0]:
model  =  make_model()
model.compile(optimizer = "adam", loss = "binary_crossentropy",
                    metrics=["accuracy"])
m_check = keras.callbacks.ModelCheckpoint(filepath = './cnn_spectrogram_mf.h5', monitor='val_acc', save_best_only=True, mode='max', verbose=1 )
hist = model.fit(X_train, y_train, 
                 batch_size=32, validation_data=(X_test,y_test),nb_epoch=100, verbose=1, shuffle = True,callbacks=[m_check] 
                )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 2034 samples, validate on 625 samples
Epoch 1/100
2034/2034 [==============================] - 47s 23ms/step - loss: 1.6091 - acc: 0.5000 - val_loss: 0.6660 - val_acc: 0.6208

Epoch 00001: val_acc improved from -inf to 0.62080, saving model to ./cnn_spectrogram_mf.h5
Epoch 2/100
2034/2034 [==============================] - 49s 24ms/step - loss: 0.8218 - acc: 0.5147 - val_loss: 0.6668 - val_acc: 0.5680

Epoch 00002: val_acc did not improve from 0.62080
Epoch 3/100
2034/2034 [==============================] - 49s 24ms/step - loss: 0.7389 - acc: 0.5005 - val_loss: 0.6531 - val_acc: 0.6496

Epoch 00003: val_acc improved from 0.62080 to 0.64960, saving model to ./cnn_spectrogram_mf.h5
Epoch 4/100
2034/2034 [==============================] - 49s 24ms/step - loss: 0.7544 - acc: 0.5221 - val_loss: 0.6941 - val_acc: 0.5104

Epoch 00004: val_acc did not improve from 0.64960
Epoch 5/100
2034/2034 [==============================] - 49s 24ms/step - loss: 0.7118 - acc: 0.5182 - val_loss: 0.

In [0]:
data2[0]['emotion']

'neu'

In [0]:
X_train = []
X_test = []
Y_train = []
Y_test = []
fs = 16e3
counter = 0
counter1 = 0
for ses_mod in data2:
    if 'impro' in ses_mod['id'] and ses_mod['emotion'] in emotions_used and ses_mod['id'][-4]=='M' :
        f, t, Sxx = signal.spectrogram(ses_mod['signal'], fs, nperseg=400,noverlap=200)
        Sxx = pad_sequence_into_array(Sxx,maxlen=300,value=0)
        
        if ses_mod['id'][:5]=="Ses05":
            counter+=1
            X_test.append(Sxx)
            Y_test.append(ses_mod['emotion'])
        else:
            counter1+=1
            X_train.append(Sxx)
            Y_train.append(ses_mod['emotion'])
        
print(counter)
print(counter1)

X_test = np.array(X_test)
X_train = np.array(X_train)
print(X_train.shape)
print(X_test.shape)

315
1009
(1009, 201, 300)
(315, 201, 300)


In [0]:
X_train = X_train.reshape(-1,201,300,1)
X_test = X_test.reshape(-1,201,300,1)

In [0]:
y  = pd.get_dummies(Y_train+Y_test)
y_train = y[0:len(Y_train)]
y_test = y[len(Y_train):]
y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
def make_model():
    in_layer = Input(shape=(201, 300,1))
    x = Conv2D(32,(5,5), activation = 'elu')(in_layer)  
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(128, (5,5))(x)                           
    x = GlobalAveragePooling2D()(x)                     
    x = Dense(64,activation='elu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(4, activation = "softmax")(x) 
    model = Model(inputs = in_layer, outputs=output_layer)
    return model

In [0]:
model  =  make_model()
model.compile(optimizer = "adam", loss = "categorical_crossentropy",
                    metrics=["accuracy"])
m_check = keras.callbacks.ModelCheckpoint(filepath = './cnn_spectrogram_male.h5', monitor='val_acc', save_best_only=True, mode='max', verbose=1 )
hist = model.fit(X_train, y_train, 
                 batch_size=32, validation_data=(X_test,y_test),nb_epoch=100, verbose=1, shuffle = True,callbacks=[m_check] 
                )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 1009 samples, validate on 315 samples
Epoch 1/100
1009/1009 [==============================] - 31s 30ms/step - loss: 4.3861 - acc: 0.3994 - val_loss: 1.5528 - val_acc: 0.4159

Epoch 00001: val_acc improved from -inf to 0.41587, saving model to ./cnn_spectrogram_male.h5
Epoch 2/100
1009/1009 [==============================] - 24s 24ms/step - loss: 1.6424 - acc: 0.4103 - val_loss: 1.1777 - val_acc: 0.4762

Epoch 00002: val_acc improved from 0.41587 to 0.47619, saving model to ./cnn_spectrogram_male.h5
Epoch 3/100
1009/1009 [==============================] - 24s 24ms/step - loss: 1.2917 - acc: 0.4757 - val_loss: 1.2221 - val_acc: 0.5016

Epoch 00003: val_acc improved from 0.47619 to 0.50159, saving model to ./cnn_spectrogram_male.h5
Epoch 4/100
1009/1009 [==============================] - 24s 24ms/step - loss: 1.1749 - acc: 0.5015 - val_loss: 1.1022 - val_acc: 0.5556

Epoch 00004: val_acc improved from 0.50159 to 0.55556, saving model to ./cnn_spectrogram_male.h5
Epoch 5/100
1009

In [6]:
X_train = []
X_test = []
Y_train = []
Y_test = []
fs = 16e3
counter = 0
counter1 = 0
for ses_mod in data2:
    if 'impro' in ses_mod['id'] and ses_mod['emotion'] in emotions_used and ses_mod['id'][-4]=='F' :
        f, t, Sxx = signal.spectrogram(ses_mod['signal'], fs, nperseg=400,noverlap=200)
        Sxx = pad_sequence_into_array(Sxx,maxlen=300,value=0)
        
        if ses_mod['id'][:5]=="Ses05":
            counter+=1
            X_test.append(Sxx)
            Y_test.append(ses_mod['emotion'])
        else:
            counter1+=1
            X_train.append(Sxx)
            Y_train.append(ses_mod['emotion'])
        
print(counter)
print(counter1)

X_test = np.array(X_test)
X_train = np.array(X_train)
print(X_train.shape)
print(X_test.shape)

310
1025
(1025, 201, 300)
(310, 201, 300)


In [0]:
X_train = X_train.reshape(-1,201,300,1)
X_test = X_test.reshape(-1,201,300,1)

In [0]:
y  = pd.get_dummies(Y_train+Y_test)
y_train = y[0:len(Y_train)]
y_test = y[len(Y_train):]
y_train = np.array(y_train)
y_test = np.array(y_test)

In [14]:
model  =  make_model()
model.compile(optimizer = "adam", loss = "categorical_crossentropy",
                    metrics=["accuracy"])
m_check = keras.callbacks.ModelCheckpoint(filepath = './cnn_spectrogram_female.h5', monitor='val_acc', save_best_only=True, mode='max', verbose=1 )
hist = model.fit(X_train, y_train, 
                 batch_size=32, validation_data=(X_test,y_test),nb_epoch=100, verbose=1, shuffle = True,callbacks=[m_check] 
                )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 1025 samples, validate on 310 samples
Epoch 1/100
1025/1025 [==============================] - 35s 35ms/step - loss: 5.0200 - acc: 0.3639 - val_loss: 1.2857 - val_acc: 0.4484

Epoch 00001: val_acc improved from -inf to 0.44839, saving model to ./cnn_spectrogram_female.h5
Epoch 2/100
1025/1025 [==============================] - 22s 21ms/step - loss: 1.5970 - acc: 0.4195 - val_loss: 1.1515 - val_acc: 0.5387

Epoch 00002: val_acc improved from 0.44839 to 0.53871, saving model to ./cnn_spectrogram_female.h5
Epoch 3/100
1025/1025 [==============================] - 22s 22ms/step - loss: 1.4108 - acc: 0.4439 - val_loss: 1.3166 - val_acc: 0.4484

Epoch 00003: val_acc did not improve from 0.53871
Epoch 4/100
1025/1025 [==============================] - 22s 22ms/step - loss: 1.4080 - acc: 0.4566 - val_loss: 1.2648 - val_acc: 0.4065

Epoch 00004: val_acc did not improve from 0.53871
Epoch 5/100
1025/1025 [==============================] - 22s 22ms/step - loss: 1.2718 - acc: 0.4527 - val_